# mrjob: Muestras

En este ejemplo vamos a ver dos conceptos importantes de mrjob: los protocolos y el paso de parámetros.  En primer lugar veremos qué son los protocolos. Mrjob asume que todos los datos son bytes delimitados por saltos de línea, y los serializa y deserializa utilizando protocolos. Cada trabajo mrjob tiene un protocolo de entrada, otro de salida y otro interno.  

Un protocolo tiene un método read() y otro write(). El read() convierte los bytes a pares de objetos Python que representan las claves y los valores. El método write() a su vez convierte un par de objetos de Python en bytes.  

El protocolo de entrada se utiliza para leer los bytes que recibe el primer paso (el primer map, o el primer reduce si el primer paso no tiene map). El protocolo de salida se utiliza para escribir la salida del último paso a bytes que se escribirán en el fichero de salida. El protocolo interno convierte la salida de un paso en la entrada del siguiente paso si el trabajo tiene más de un paso. 

El protocolo de entrada por defecto es RawValueProtocol, que simplemente lee una línea como un str. Por tanto, el primer paso del trabajo mrjob ve (None, línea) para cada línea de entrada. 

Los protocolos interno y de salida por defecto en mrjob son ambos JSONProtocol, que leen y escriben cadenas JSON separadas por un tabulador (por defecto, Hadoop Streaming utiliza el tabulador para separar claves y valores en una línea cuando ordena los datos). 

Para entenderlo de forma sencilla, se utiliza RawValueProtocol para leer o escribir líneas de texto en bruto, y JSONProtocol para leer o escribir pares clave-valor donde la clave y el valor son JSON.  

En el siguiente enlace se muestran los diferentes protocolos que permite mrjob: 

https://pythonhosted.org/mrjob/protocols.html#module-mrjob.protocol 

En este ejercicio vemos también la forma de pasar parámetros de entrada a nuestro trabajo mrjob. El ejercicio consiste en desarrollar un código que tome como entrada un fichero de texto y devuelva como salida una muestra de sus líneas, tantas como se indique como parámetro. 

In [1]:
! mkdir -p mrjob/muestras

In [2]:
import os
os.chdir("/media/notebooks/mrjob/muestras")

In [3]:
! pwd

/media/notebooks/mrjob/muestras


In [4]:
%%writefile mrjob-ejercicio.py
import random
import sys

from mrjob.job import MRJob
from mrjob.protocol import RawValueProtocol, ReprProtocol

FACTOR_MUESTREO = 1.2

class MRMuestreo(MRJob):
    
    # Utilizamos RawValueProtocol para la salida para devolver lineas en bruto  
    # en lugar de pares (clave, valor) 
    OUTPUT_PROTOCOL = RawValueProtocol 
    # OUTPUT_PROTOCOL = ReprProtocol # Prueba a comentar y descomentar estas lineas para ver el efecto sobre la salida
        
    def configure_args(self):
        super(MRMuestreo, self).configure_args()
        self.add_passthru_arg(
            '--sample-size',
            type=float,
            help='Numero muestras que retornar.'
        )
        self.add_passthru_arg(
            '--expected-length',
            type=float,
            help=("Numero of entries you expect in the log. If not specified,"
                  " we'll pass every line to the reducer.")
        )

    def load_args(self, args):
        super(MRMuestreo, self).load_args(args)

        if self.options.sample_size is None:
            self.option_parser.error('You must specify the --sample-size')
        else:
            self.sample_size = self.options.sample_size

        # Si hemos incluido una longitud estimada, podemos calcular la  
        # probabilidad de muestreo para el mapper, de forma que el  
        # reducer no tiene que recibir todas las entradas 
        if self.options.expected_length is None:
            self.sampling_probability = 1.
        else:

            self.sampling_probability = (self.sample_size *
                                         FACTOR_MUESTREO /
                                         self.options.expected_length)

    def mapper(self, _, line):
        """ 
        Para cada linea, con probabilidad self.sampling_probability, 
        se emite una clave None, y (random, linea) como valor, de forma que 
        los valores se ordenen aleatoriamente y se envien a un unico reducer 

        Argumentos: 
            line - linea en bruto 
        Emite: 
            key - None 
            value - (random seed, line) 
        """ 

        if random.random() < self.sampling_probability:
            seed = '%20i' % random.randint(0, sys.maxsize)
            yield None, (seed, line)

    def reducer(self, _, values):
        """ 
        Los valores tienen asignado un numero aleatorio,  
        Llegaran en orden aleatorio, asi que se emiten las primeras n lineas. 
        Argumentos: 
            values - generador de pares (random_seed, linea)  
        Emite: 
            key - None 
            value - muestra aleatoria de lineas del fichero 
        """ 

        for line_num, (seed, line) in enumerate(values):
            yield None, line

            # enumerate() comienza en 0, asi que sumamos 1
            if line_num + 1 >= self.sample_size:
                break


if __name__ == '__main__':
    MRMuestreo.run()

Overwriting mrjob-ejercicio.py


In [5]:
# Averiguamos cuantas lineas tiene el fichero para que sirve como parámetro del programa
! wc -l /media/notebooks/marktwain.txt 

302272 /media/notebooks/marktwain.txt


In [9]:
! python mrjob-ejercicio.py /media/notebooks/marktwain.txt \
--sample-size 3 --expected-length 302272 > ouputlocal

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/mrjob-ejercicio.root.20190812.105554.161994
Running step 1 of 1...
job output is in /tmp/mrjob-ejercicio.root.20190812.105554.161994/output
Streaming final output from /tmp/mrjob-ejercicio.root.20190812.105554.161994/output...
Removing temp directory /tmp/mrjob-ejercicio.root.20190812.105554.161994...


In [10]:
! tail ouputlocal

rage into which they lashed themselves; and, secondly, the puerility of
and then it solidified itself, and you could have walked upon
once, and it went by very dim to the sight and floated noiseless through


In [12]:
! hdfs dfs -rm /tmp/carpeta/mrjob-muestras-output/*
! hdfs dfs -rmdir /tmp/carpeta/mrjob-muestras-output

Deleted /tmp/carpeta/mrjob-muestras-output/_SUCCESS
Deleted /tmp/carpeta/mrjob-muestras-output/part-00000


In [13]:
! python mrjob-ejercicio.py hdfs:///tmp/carpeta/marktwain.txt --sample-size 3 \
--expected-length 302272 --output-dir hdfs:///tmp/carpeta/mrjob-muestras-output \
-r hadoop --python-bin /opt/anaconda/bin/python3.7

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/lib/hadoop/bin...
Found hadoop binary: /usr/lib/hadoop/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /usr/lib/hadoop...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/mrjob-ejercicio.root.20190812.105755.193365
uploading working dir files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio.root.20190812.105755.193365/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/mrjob-ejercicio.root.20190812.105755.193365/files/
Running step 1 of 1...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob5467777747702039131.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.21.0.3:8032
  Connecting to ResourceManager at yarnmaster/172.21.0.3:8032
  Total 

In [14]:
! hdfs dfs -tail /tmp/carpeta/mrjob-muestras-output/part-00000


     elephant went west.  Shall now shadow him in that direction.	
of monotony, and keep it up till the monotonies ran out, if it was a	
I said I did not know, but we could try a dog and see.  So he sent	
